##  Inspecting the transfusion.data file

In [10]:
# Print out the first 5 lines from the transfusion.data file
!head -n5 transfusion.data

## Loading the blood donations data
We now know that we are working with a typical CSV file. We proceed to loading the data into memory.

In [11]:
# Import pandas
import pandas as pd
transfusion = pd.read_csv('transfusion.data')
transfusion.head()

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0


<li>R (Recency - months since the last donation)</li>
<li>F (Frequency - total number of donation)</li>
<li>M (Monetary - total blood donated in c.c.)</li>
<li>T (Time - months since the first donation)</li>
<li>a binary variable representing whether he/she donated blood in March 2007 (1 stands for donating blood; 0 stands for not donating blood)</li>
</ul>
<p>It looks like every column in our DataFrame has the numeric type, which is exactly what we want when building a machine learning model. Let's verify our hypothesis.</p>

## Summary of DataFrame

In [13]:
transfusion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 5 columns):
 #   Column                                      Non-Null Count  Dtype
---  ------                                      --------------  -----
 0   Recency (months)                            748 non-null    int64
 1   Frequency (times)                           748 non-null    int64
 2   Monetary (c.c. blood)                       748 non-null    int64
 3   Time (months)                               748 non-null    int64
 4   whether he/she donated blood in March 2007  748 non-null    int64
dtypes: int64(5)
memory usage: 29.3 KB


## Creating the Target Column

We are aiming to predict the value in <code>whether he/she donated blood in March 2007</code> column. Let's rename it to <code>target</code> so that it's more convenient to work with.

In [15]:
# Renaming target column as 'target' for brevity 
transfusion.rename(
    columns={'whether he/she donated blood in March 2007': 'target'},
    inplace=True
)

transfusion['target'].head(2)

0    1
1    1
Name: target, dtype: int64

## Checking "Target incidence"
We want to predict whether or not the same donor will give blood the next time the vehicle comes to campus. The model for this is a binary classifier, meaning that there are only 2 possible outcomes:</p>
<ul>
<li><code>0</code> - the donor will not give blood</li>
<li><code>1</code> - the donor will give blood</li>
</ul>
<p>Target incidence is defined as the number of cases of each individual target value in a dataset. That is, how many 0s are in the target column compared to the number of 1s. The Target incidence gives us an idea of how balanced (or imbalanced) the dataset is.</p>

In [16]:
# Printing the target incidence proportions, rounding output to 3 decimal places
transfusion.target.value_counts(normalize=True).round(3)

0    0.762
1    0.238
Name: target, dtype: float64

## Splitting the dataset into train and test sets
<p>By investigating the Target incidence in the previous chunk of code, we are informed that <code>0</code>s appear 76% of the time. I'd like to keep the same structure in both my train and test sets, i.e., both datasets must have 0 target incidence of 76%. 

In [17]:
# Import train_test_split method
from sklearn.model_selection import train_test_split

# Split transfusion DataFrame into
# X_train, X_test, y_train and y_test datasets,
# stratifying on the `target` column
X_train, X_test, y_train, y_test = train_test_split(
    transfusion.drop(columns='target'),
    transfusion.target,
    test_size=0.25,
    random_state=42,
    stratify=transfusion.target
)

X_train.head(2)

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months)
334,16,2,500,16
99,5,7,1750,26


## Selecting a model through TPOT (Tree-based Pipeline Optimization Tool)
TPOT will automatically explore hundreds of possible pipelines to find the best one for our dataset.The outcome of this search will be a scikit-learn pipeline, meaning it will include any pre-processing steps as well as the model. TPOT will help me zero in on one model that we can then explore and optimize further.

In [18]:
# Import TPOTClassifier and roc_auc_score
from tpot import TPOTClassifier
from sklearn.metrics import roc_auc_score


# Instantiate TPOTClassifier
tpot = TPOTClassifier(
    generations=5,
    population_size=20,
    verbosity=2,
    scoring='roc_auc',
    random_state=42,
    disable_update_check=True,
    config_dict='TPOT light'
)
tpot.fit(X_train, y_train)

# AUC score for tpot model
tpot_auc_score = roc_auc_score(y_test, tpot.predict_proba(X_test)[:, 1])
print(f'\nAUC score: {tpot_auc_score:.4f}')

# Print best pipeline steps
print('\nBest pipeline steps:', end='\n')
for idx, (name, transform) in enumerate(tpot.fitted_pipeline_.steps, start=1):
    # Print idx and transform
    print(f'{idx}.{transform}')

Generation 1 - Current best internal CV score: 0.7422459184429089


Generation 2 - Current best internal CV score: 0.7422459184429089


Generation 3 - Current best internal CV score: 0.7422459184429089


Generation 4 - Current best internal CV score: 0.7422459184429089


Generation 5 - Current best internal CV score: 0.7456308339276876

Best pipeline: MultinomialNB(Normalizer(input_matrix, norm=l2), alpha=0.001, fit_prior=True)

AUC score: 0.7637

Best pipeline steps:
1.Normalizer(copy=True, norm='l2')
2.MultinomialNB(alpha=0.001, class_prior=None, fit_prior=True)


<p></p>
<p></p>
<p></p>
<p></p>
<p>TPOT picked <code>LogisticRegression</code> as the best model for the dataset. It is worth keeping in mind that this was with no pre-processing steps. With this in mind, I'd say that the model achieving an AUC score of 0.7637 is definitely a great starting point. Let's see if we can make it better.</p>
<p>One of the assumptions for linear regression models is that the data and the features we are giving it are related in a linear fashion, or can be measured with a linear distance metric. If a feature in our dataset has a high variance that's an order of magnitude or more greater than the other features, this could impact the model's ability to learn from other features in the dataset.</p>

## Checking for Variance

Correcting for high variance is called normalization. It is one of the possible transformations you do before training a model. Let's check the variance to see if such transformation is needed.</p>

In [19]:
X_train.var().round(3)

Recency (months)              66.929
Frequency (times)             33.830
Monetary (c.c. blood)    2114363.700
Time (months)                611.147
dtype: float64

Monetary (c.c. blood)'s variance is very high in comparison to any other column in the dataset. This means that, unless accounted for, this feature may be given more weight than is appropriate, by the model (i.e., be seen as more important than other features present)

## Correcting for high variance through Log Normalization

In [21]:
# Import numpy
import numpy as np

# Copy X_train and X_test into X_train_normed and X_test_normed
X_train_normed, X_test_normed = X_train.copy(), X_test.copy()

# Specify which column to normalize
col_to_normalize = 'Monetary (c.c. blood)'

# Log normalization
for df_ in [X_train_normed, X_test_normed]:
    # Add log normalized column
    df_['monetary_log'] = np.log(df_[col_to_normalize])
    # Drop the original column
    df_.drop(columns=col_to_normalize, inplace=True)

# Check the variance for X_train_normed
# ... YOUR CODE FOR TASK 9 ...'
X_train_normed.var().round(3)

Recency (months)      66.929
Frequency (times)     33.830
Time (months)        611.147
monetary_log           0.837
dtype: float64

Even though "Time (months)" now has the largest variance, I will leave it for now since the order of magnitude of its variance is not as bad as the "Monetary (c.c. blood)" feature was (Just kidding, I'm just giving myself a higher chance of improving my model in the future haha) so we'll leave it as is.

## Training the Linear Regression Model (Finally!)

In [22]:
# Importing modules
from sklearn import linear_model

# Instantiate LogisticRegression
logreg = linear_model.LogisticRegression(
    solver='liblinear',
    random_state=42
)

# Train the model
logreg.fit(X_train_normed, y_train)

# AUC score for tpot model
logreg_auc_score = roc_auc_score(y_test, logreg.predict_proba(X_test_normed)[:, 1])
print(f'\nAUC score: {logreg_auc_score:.4f}')


AUC score: 0.7890


##### After normalizing a single feature, the AUC score of my model mproved by 3.31%! I might have to add TPOT to my data science toolkit.